# Metadata First Attempt
This was the first attempt to build metadata tables for the coldstart model

In [10]:
#!/usr/bin/env python
# -*- encoding: utf-8 -*-

# USAGE:
#   python Lab1.py Sample_Song_Dataset.db

import sys
import sqlite3
import time
import pandas as pd

In [4]:
# The database file should be given as the first argument on the command line
db_file = 'track_metadata.db'

In [8]:
#GET THE 
with sqlite3.connect(db_file) as conn:
    # We use a "cursor" to mark our place in the database.
    # We could use multiple cursors to keep track of multiple
    # queries simultaneously.
    cursor = conn.cursor()

    # This query counts the number of tracks from the year 1998
    cursor.execute('SELECT * FROM songs LIMIT 3')

    for row in cursor:
        print(row)

('TRMMMYQ128F932D901', 'Silent Night', 'SOQMMHC12AB0180CB8', 'Monster Ballads X-Mas', 'ARYZTJS1187B98C555', '357ff05d-848a-44cf-b608-cb34b5701ae5', 'Faster Pussy cat', 252.05506, 0.649822100201, 0.394031892714, 2003, 7032331, -1, 0)
('TRMMMKD128F425225D', 'Tanssi vaan', 'SOVFVAK12A8C1350D9', 'Karkuteillä', 'ARMVN3U1187FB3A1EB', '8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9', 'Karkkiautomaatti', 156.55138, 0.439603966677, 0.356992107756, 1995, 1514808, -1, 0)
('TRMMMRX128F93187D9', 'No One Could Ever', 'SOGTUKN12AB017F4F1', 'Butter', 'ARGEKB01187FB50750', '3d403d44-36ce-465c-ad43-ae877e65adc4', 'Hudson Mohawke', 138.97098, 0.643680572058, 0.437503836595, 2006, 6945353, -1, 0)


In [9]:
#GET THE SCHEMA FOR THE SONGS TABLE
with sqlite3.connect(db_file) as conn:
    
    cursor = conn.cursor()
    cursor.execute("""SELECT sql 
                    FROM sqlite_master 
                    WHERE name = 'songs';""")

    for row in cursor:
        print(row)

('CREATE TABLE songs (track_id text PRIMARY KEY, title text, song_id text, release text, artist_id text, artist_mbid text, artist_name text, duration real, artist_familiarity real, artist_hotttnesss real, year int, track_7digitalid int, shs_perf int, shs_work int)',)


In [12]:
#TURN THIS TABLE INTO A CSV PARQUET FILE
with sqlite3.connect(db_file) as conn:
    
    df = pd.read_sql('SELECT * from songs', conn)
    df.to_parquet('items_meta.parquet', index = False)

In [14]:
items = pd.read_parquet('items_meta.parquet')

In [19]:
#get most played
items.loc[items.track_id == 'TRDMBIJ128F4290431']

track_id          title             song_id  \
808198  TRDMBIJ128F4290431  Sehr kosmisch  SOFRQTD12A81C233C0   

                   release           artist_id  \
808198  Musik von Harmonia  AR0IVTL1187B9AD520   

                                 artist_mbid artist_name   duration  \
808198  122d63fc-8671-43e4-9752-34e846d62a9c    Harmonia  655.77751   

        artist_familiarity  artist_hotttnesss  year  track_7digitalid  \
808198            0.505949           0.848826     0           2436752   

        shs_perf  shs_work  
808198        -1         0

In [41]:
q75 = items.duration.quantile(q=0.75)
q50 = items.duration.quantile(q=0.5)
q25 = items.duration.quantile(q=0.25)



#split into 4 files?
df1 = items.loc[items.duration >= q75]
df2 = items.loc[(items.duration >= q50) & (items.duration < q75)]
df3 = items.loc[(items.duration >= q25) & (items.duration < q50)]
df4 = items.loc[items.duration < q25]

In [42]:
dfs = [df1,df2,df3,df4]

tot = 0
for d in dfs:
    tot += d.shape[0]
    print(d.shape[0])
    
tot

250029
249999
249989
249983


1000000

In [43]:
df1.to_parquet('items_meta_1.parquet')
df2.to_parquet('items_meta_2.parquet')
df3.to_parquet('items_meta_3.parquet')
df4.to_parquet('items_meta_4.parquet')